In [51]:
import numpy as np
import json_tricks
import os

answer = {}
inputs = json_tricks.load('.inputs.json')

FileNotFoundError: [Errno 2] No such file or directory: '.inputs.json'

# Task

You are given a linear hyperspace by 2 entities:
- a vector that belongs to the hyperspace $\mathbf x_0$
- a normal vector to this hyperspace $\mathbf w$

So that the hyperplane is given by equation:

$\left<\mathbf w, \mathbf x - \mathbf x_0\right> = 0$

You are also given another vector $\mathbf y$. 

Find distance from this vector to the given hyperplane.

In [ ]:
def distance_to_hyperplane_1(y, w, x0):
    y = np.array(y)
    w = np.array(w)
    x0 = np.array(x0)
    y_minus_x0 = y - x0
    dot_product = np.dot(w, y_minus_x0)
    norm_w = np.linalg.norm(w)
    distance = np.abs(dot_product) / norm_w
    return distance.item() 

In [ ]:
answer['task1'] = []
if 'task1' in inputs:
    for one_input in inputs['task1']:
        answer['task1'].append(
            distance_to_hyperplane_1(**one_input))

# Task

You are given a linear hyperspace by 2 entities:
- a vector that belongs to the hyperspace $\mathbf x_0$
- a set of direction vectors for this hyperspace $\mathbf a_1, \dots, \mathbf a_K$ in form of column-matrix 
    
    $A = \begin{bmatrix}
    | & | & & | \\
    \mathbf a_1 & \mathbf a_2 & \dots & \mathbf a_K \\
    | & | & & | \\
    \end{bmatrix}$

So that the hyperplane is given by equation:

$\mathbf x_0 + \sum_{k=1}^K \alpha_k \mathbf a_k = \mathbf 0$

You are also given another vector $\mathbf y$. 

Find distance from this vector to the given hyperplane.

In [ ]:
def distance_to_hyperplane_2(y, A, x0):
    y = np.array(y)
    A = np.array(A)
    x0 = np.array(x0)
    
    # Compute y - x0
    y_minus_x0 = y - x0
    
    # Compute the projection matrix P = A (A^T A)^-1 A^T
    # Handle cases where A^T A is not invertible (use pseudo-inverse)
    P = A @ np.linalg.pinv(A.T @ A) @ A.T
    
    # Compute the residual
    residual = y_minus_x0 - P @ y_minus_x0
    
    # Distance is the norm of the residual
    distance = np.linalg.norm(residual)
    
    return distance


In [ ]:
answer['task2'] = []
if 'task2' in inputs:
    for one_input in inputs['task2']:
        answer['task2'].append(
            distance_to_hyperplane_2(**one_input))

# Task

You are given 2 linear hyperspaces, each defined by 2 entities:
- a vector that belongs to the hyperspace $\mathbf x_0$
- a set of direction vectors for this hyperspace $\mathbf a_1, \dots, \mathbf a_K$ in form of column-matrix 

So that each of the hyperplanes is given by equation:

$\mathbf x_0 + \sum_{k=1}^K \alpha_k \mathbf a_k = \mathbf 0$

Find distance between these hyperplanes.

In [ ]:
def distance_to_hyperplane_3(A, x0, B, y0):

    # Convert inputs to numpy arrays
    A = np.array(A, dtype=float)
    x0 = np.array(x0, dtype=float)
    B = np.array(B, dtype=float)
    y0 = np.array(y0, dtype=float)
    
    # Ensure matrices are 2D
    if A.ndim == 1:
        A = A.reshape(-1, 1)
    if B.ndim == 1:
        B = B.reshape(-1, 1)
    
    # Compute difference between base points
    diff = y0 - x0
    
    # Check if hyperplanes are parallel
    # Method 1: Compare null spaces
    def nullspace(A, atol=1e-13):
        u, s, vh = np.linalg.svd(A)
        tol = max(atol, s.max() * max(A.shape) * np.finfo(s.dtype).eps)
        nnz = (s >= tol).sum()
        ns = vh[nnz:].conj().T
        return ns
    
    ns_A = nullspace(A.T)
    ns_B = nullspace(B.T)
    
    # If null spaces are not equal, hyperplanes intersect
    if ns_A.shape[1] > 0 and ns_B.shape[1] > 0:
        # Check if null spaces are parallel
        if np.linalg.matrix_rank(np.hstack((ns_A, ns_B))) > max(ns_A.shape[1], ns_B.shape[1]):
            return 0.0
    
    # If we get here, hyperplanes are parallel
    # Compute orthogonal complement of A's column space
    Q, _ = np.linalg.qr(A)
    
    # Project difference vector onto orthogonal complement
    if Q.size > 0:
        proj = Q @ (Q.T @ diff)
        residual = diff - proj
    else:
        residual = diff
    
    distance = np.linalg.norm(residual)
    
    return distance

In [ ]:
answer['task3'] = []
if 'task3' in inputs:
    for one_input in inputs['task3']:
        answer['task3'].append(
            distance_to_hyperplane_3(**one_input))

In [ ]:
json_tricks.dump(answer, '.answer.json')

'{"task1": [], "task2": [], "task3": []}'